# Tip5

In [ ]:
p = 2**64-2**32+1
Fp = GF(p)

r = 2**8+1
Fr = GF(r)

## S-boxes

In [ ]:
## Define the S-box T.

def T(x):
    """
    Input:
        x:Fp
    Output:
        T(x):Fp
    """
    return x**7

def T_inv(x):
    """
    Inverse of T.
    """
    return x**10540996611094048183

In [ ]:
## Define the S-box S.

R = Fp(2**64)

def sigma(x):
    """
    Input:
        x:Fp
    Output:
        x_list:Fr⁸, decomposition of x in base 256
    """
    b = 256
    x_int = ZZ(x)
    x_list = []
    for i in range(8):
        xi = Fr(x_int % b)
        x_list.append(xi)
        x_int //= 256
    return x_list

def L(xi):
    """
    Input:
        xi:Fr
    Output:
        L(xi):Fr
    """
    return (xi+1)**3-1

def L8(x_list):
    """
    Input:
        x_list:Fr⁸
    Output:
        L8(x_list):Fr⁸, L applied elementwise
    """
    x_L8 = [L(xi) for xi in x_list]
    return x_L8

def L_inv(xi):
    """
    Inverse of L.
    """
    return (xi+1)**171-1

def L8_inv(x_list):
    """
    Inverse of L8.
    """
    x_L8_inv = [L_inv(xi) for xi in x_list]
    return x_L8_inv

def rho(x_list):
    """
    Input:
        x_list:Fr⁸
    Output:
        x:Fp, inverse operation of sigma
    """
    x = Fp(0)
    b = 256
    for i in range(8):
        x += Fp(x_list[i])*b**i
    return x
    

def S(x):
    """
    Input:
        x:Fp
    Output:
        S(x):Fp, output of the S-box S
    """
    y = 1/R * rho(L8(sigma(R*x)))
    return y

def S_inv(x):
    """
    Inverse of S.
    """
    y = 1/R * rho(L8_inv(sigma(R*x)))
    return y

## Linear layer

In [ ]:
## Linear part of Tip5.

Mcol =  vector(Fp, [
    61402, 1108, 28750, 33823, 7454, 43244, 53865, 12034, 56951, 27521, 41351, 40901, 12021, 59689, 26798, 17845
])
M = matrix.circulant(Mcol).transpose()

## Round permutation and f permutation

In [ ]:
## Compute the f permutation

def f_round(X, round_constant):
    """
    Input:
        X:Fp^16, state of the permutation
        round_constant:Fp^16, constant added to the round
    Output:
        Y:Fp^16, state after one round
    """
    Y = zero_vector(Fp, 16)
    for i in range(4):
        Y[i] = S(X[i])
    for i in range(4,16):
        Y[i] = T(X[i])
    
    Y = M*Y + round_constant
    
    return Y

def f_round_inv(Y, round_constant):
    """
    Inverse of a round of f.
    """
    M_inv = M.inverse()
    X = M_inv*(Y - round_constant)
    
    for i in range(4):
        X[i] = S_inv(X[i])
    for i in range(4,16):
        X[i] = T_inv(X[i])
    
    return X


def f_inv(Y, round_constants):
    """
    Input:
        X:Fp^16, state of the permutation
        round_constant:Fp^16, constant added to the round
    Output:
        Y:Fp^16, state after one round
    """
    X = Y
    for round_constant in round_constants[::-1]:
        X = f_round_inv(X, round_constant)
    
    return X

def f(X, round_constants):
    """
    Input:
        X:Fp^16, state of the permutation
        round_constant:Fp^16, constant added to the round
    Output:
        Y:Fp^16, state after one round
    """
    Y = X
    for round_constant in round_constants:
        Y = f_round(Y, round_constant)
    
    return Y

## Round constants

In [ ]:
## Add round constants

round_constants = [
    vector(Fp, [
        13630775303355457758,
        16896927574093233874,
        10379449653650130495,
        1965408364413093495,
        15232538947090185111,
        15892634398091747074,
        3989134140024871768,
        2851411912127730865,
        8709136439293758776,
        3694858669662939734,
        12692440244315327141,
        10722316166358076749,
        12745429320441639448,
        17932424223723990421,
        7558102534867937463,
        15551047435855531404
            ]
        ),
    vector(Fp, [
        17532528648579384106,
        5216785850422679555,
        15418071332095031847,
        11921929762955146258,
        9738718993677019874,
        3464580399432997147,
        13408434769117164050,
        264428218649616431,
        4436247869008081381,
        4063129435850804221,
        2865073155741120117,
        5749834437609765994,
        6804196764189408435,
        17060469201292988508,
        9475383556737206708,
        12876344085611465020
            ]
        ),
    vector(Fp, [
        13835756199368269249,
        1648753455944344172,
        9836124473569258483,
        12867641597107932229,
        11254152636692960595,
        16550832737139861108,
        11861573970480733262,
        1256660473588673495,
        13879506000676455136,
        10564103842682358721,
        16142842524796397521,
        3287098591948630584,
        685911471061284805,
        5285298776918878023,
        18310953571768047354,
        3142266350630002035
            ]
        ),
    vector(Fp, [
        549990724933663297,
        4901984846118077401,
        11458643033696775769,
        8706785264119212710,
        12521758138015724072,
        11877914062416978196,
        11333318251134523752,
        3933899631278608623,
        16635128972021157924,
        10291337173108950450,
        4142107155024199350,
        16973934533787743537,
        11068111539125175221,
        17546769694830203606,
        5315217744825068993,
        4609594252909613081
            ]
        ),
    vector(Fp, [
        3350107164315270407,
        17715942834299349177,
        9600609149219873996,
        12894357635820003949,
        4597649658040514631,
        7735563950920491847,
        1663379455870887181,
        13889298103638829706,
        7375530351220884434,
        3502022433285269151,
        9231805330431056952,
        9252272755288523725,
        10014268662326746219,
        15565031632950843234,
        1209725273521819323,
        6024642864597845108
            ]
        )
]


## The Tip5 hash function

In [ ]:
def Tip5(input_vec):
    """
    Implementation of the 10-to-5 Tip5 hash function.
    
    Input:
        input_vec: Fp^10.
    Output:
        output: Fp^5.
    """
    X = zero_vector(Fp, 16)
    for i in range(10):
        X[i] = input_vec[i]
    for i in range(10,16):
        X[i] = Fp(1)
    
    Y = f(X, round_constants)
    output = Y[:5]
    return output


# Tip5 mini

In [ ]:
p_mini = 2**8-2**4+1
Fp_mini = GF(p_mini)

r_mini = 2**2+1
Fr_mini = GF(r_mini)

## S-boxes

In [ ]:
def T_mini(x):
    return x**7

def T_mini_inv(x):
    return x**103

In [ ]:
R_mini = Fp_mini(2**8)

def sigma_mini(x):
    b = 4
    x_int = ZZ(x)
    x_list = []
    for i in range(4):
        xi = Fr_mini(x_int % b)
        x_list.append(xi)
        x_int //= b
    return x_list

def L_mini(xi):
    return (xi+1)**3-1

def L_mini_inv(xi):
    return (xi+1)**11-1

def L4_mini(x_list):
    x_L2 = [L_mini(xi) for xi in x_list]
    return x_L2

def L4_mini_inv(x_list):
    x_L2 = [L_mini_inv(xi) for xi in x_list]
    return x_L2

def rho_mini(x_list):
    x = Fp_mini(0)
    b = 4
    for i in range(4):
        x += Fp_mini(x_list[i])*b**i
    return x
    

def S_mini(x):
    y = 1/R_mini * rho_mini(L4_mini(sigma_mini(R_mini*x)))
    return y

def S_mini_inv(x):
    y = 1/R_mini * rho_mini(L4_mini_inv(sigma_mini(R_mini*x)))
    return y

## Linear Layer

In [ ]:
def gen_MDS(t):
    """
    Generate a circulant MDS matrix when the number of branches t divides p_mini - 1.
    """
    alpha = Fp_mini.primitive_element()
    a = alpha**((p_mini-1)//t)
    values = [a**i for i in range(t)]

    b = Fp_mini(1)
    while b in values:
        b += 1

    vec = [1/(1-b*a**i) for i in range(16)]
    M_mini = matrix.circulant(vec)
    return M_mini


In [ ]:
M_mini = gen_MDS(16)

## Round permutation and f permutation

In [ ]:
def f_round_mini(X, round_constant):
    Y = zero_vector(Fp_mini, 16)
    for i in range(4):
        Y[i] = S_mini(X[i])
    for i in range(4,16):
        Y[i] = T_mini(X[i])
    
    Y = M_mini*Y + round_constant
    
    return Y

def f_round_mini_inv(X, round_constant):
    Y = zero_vector(Fp_mini, 16)
    Y = M_mini.inverse()*(X-round_constant)
    
    for i in range(4):
        Y[i] = S_mini_inv(Y[i])
    for i in range(4,16):
        Y[i] = T_mini_inv(Y[i])
    
    return Y

def f_mini(X, round_constants):
    Y = X
    for round_constant in round_constants:
        Y = f_round_mini(Y, round_constant)
    return Y

def f_mini_inv(X, round_constants):
    Y = X
    for round_constant in round_constants[::-1]:
        Y = f_round_mini_inv(Y, round_constant)
    return Y

## Round constants

In [ ]:
# round_constants_mini = [vector(Fp_mini, [randrange(241) for i in range(16)]) for n_round in range(5)]
# round_constants_mini

In [ ]:
round_constants_mini = [vector(Fp_mini, [22, 134, 31, 48, 218, 188, 34, 0, 150, 82, 200, 149, 45, 60, 148, 57]),
 vector(Fp_mini, [85, 108, 112, 159, 199, 40, 105, 149, 166, 115, 47, 192, 115, 197, 209, 239]),
 vector(Fp_mini, [20, 64, 184, 26, 94, 173, 162, 137, 179, 136, 201, 165, 119, 103, 32, 53]),
 vector(Fp_mini, [57, 239, 208, 219, 40, 215, 117, 55, 203, 56, 176, 6, 75, 183, 46, 195]),
 vector(Fp_mini, [96, 54, 175, 95, 112, 185, 130, 114, 75, 139, 206, 85, 211, 140, 48, 105])]

In [ ]:
def Tip5_mini(input_vec):
    """
    Implementation of the 10-to-5 Tip5 hash function.
    
    Input:
        input_vec: Fp^10.
    Output:
        output: Fp^5.
    """
    X = zero_vector(Fp_mini, 16)
    for i in range(10):
        X[i] = input_vec[i]
    for i in range(10,16):
        X[i] = Fp_mini(1)
    
    Y = f_mini(X, round_constants_mini)
    output = Y[:5]
    return output


## Solution to 1-CICO problem, 3-out-of-4 rounds

Give an input vector $\vec{x} = (\dots , 0)$ such that $f_3 (\vec{x}) = (\vec{a}\dots)$, where $\vec{a}\in \vec{v}^\bot \subset \mathbb{F}_p^5$, with some well-chosen $\vec{v}$.

In [ ]:
K.<x1,x2,x3,x4> = PolynomialRing(Fp, order="degrevlex")

FpZ.<z> = PolynomialRing(Fp)

In [ ]:
# The functions to do the collision attack on 4 rounds, c=5.

def complementary(indices, n):
    """
    Input:
        indices: set of integer i<n
        n: integer
    Output:
        comp: list of i<n such that i not in indices.
    """
    comp = []
    
    for i in range(n):
        if i in indices:
            continue
        else:
            comp.append(i)
    return comp


def orthogonal_vector(M, indices_a):
    """
    Input:
        M: 16x16 matrix, linear layer.
        indices: indices controlled in the vector a.
    Output:
        a: length 16 vector in Fp.
    """
    M_inv = M.inverse()
    M_inv_sub = M_inv[11:, indices_a]
    
    # basis of the vector space in which the 5 words in the capacity and the 4 wires before S-boxes are fixed.
    basis = matrix(K, M_inv_sub.right_kernel().basis()).transpose()
    
    vec = basis*vector(K, [1,x1,x2,x3,x4])
    a = zero_vector(K, 16)
    
    # shift the coordinates to get back to Fp^16.
    indices_b = complementary(indices_a, 16)
    index = 0
    for j in range(len(indices_a)):
        while index in indices_b:
            index += 1
        a[index] = T(vec[j])
        index += 1

    a = M*a
    return a

def fix_constants(M, round_constant1, round_constant2, indices_b, target):
    """
    Choose the constant of the affine subspace.
    
    Input:
        M:16x16 matrix, linear layer.
        round_constant1: length 16 vector in Fp, constants of the first round.
        indices: the wires we can control.
        target: the fixed values in the capacity of the hash function (in Fp^c).
    Output:
        b: length 16 vector in Fp.
    """
    # Describe the linear system defining b.
    c = len(target)
    M_inv = M.inverse()
    target_after_affine_layer = (M_inv * round_constant1)[16-c:] + target
    M_inv_sub = M_inv[16-c:, indices_b]
    constant = M_inv_sub.solve_right(target_after_affine_layer)
    
    # put the values obtained in the support of b.
    b = zero_vector(Fp, 16)
    index = 0
    indices_a = complementary(indices_b, 16)
    for j in range(len(indices_b)):
        while index in indices_a:
            index += 1
        if index < 4:
            b[index] = S(constant[j])
        else:
            b[index] = T(constant[j])
        index += 1
    
    # Get to the end of round 2.
    b = M*b + round_constant2
    
    return b

def attack_4_rounds(M, round_constants, a, indices_b, target):
    """
    Knowing an attack vector a with support indices, 
    gets an input with capacity target whose output lies in the right hyperplane.
    
    Input:
        M:16x16 matrix, linear layer.
        round_constants: 4 length 16 vectors in Fp.
        a: length 16 vector in Fp.
        indices: list of integers.
        target: the fixed values in the capacity of the hash function (in Fp^c).
    Output:
        x: length 16 vector in Fp.
    """
    # The vector we use to cancel out the last S-box layer.
    orth_vec = M[:5, :4].left_kernel().basis()[0]
    
    # We can fix the first value of the target (in case the polynomial has no root).
    x0 = 0
    while True:
        list_target = [x0] + list(target)
        # Apply S-boxes of first round.
        cur_target = vector(Fp, list_target)
        for i in range(6):
            cur_target[i] = cur_target[i]**7
        
        b = fix_constants(M, round_constants[0], round_constants[1], indices_b, cur_target)
        poly_1 = a*z + b
        poly_2 = zero_vector(FpZ, 16)
        for i in range(4):
            poly_2[i] = S(poly_1[i])
        for i in range(4, 16):
            poly_2[i] = poly_1[i]**7
        poly_2 = M*poly_2 + round_constants[2]
        # State after round 3.
        
        poly_3 = zero_vector(FpZ, 16)
        for i in range(4, 16):
            poly_3[i] = poly_2[i]**7
        poly_3 = M*poly_3 + round_constants[3]
        # State after round 4 (not taking into account the first 4 wires).
        
        # Solve the equation to lie in the hyperplane.
        equation = poly_3[:5].dot_product(orth_vec)
        try: 
            z_sol = equation.roots()[0][0]
            x_round_1 = poly_1(z_sol)
            x = f_inv(x_round_1, round_constants[:2])
            return x
        except:   
            x0 += 1
        

## Integral attack on the full round permutation

In [ ]:
def get_a(M):
    """
    Get a suitable vector for integral attack.
    """
    a_dec = M.inverse()[:4,4:].right_kernel()[1]
    a = zero_vector(Fp_mini, 16)
    for i in range(12):
        a[i+4] = a_dec[i]
    a = M.inverse()*a
    for i in range(16):
        a[i] = T_mini_inv(a[i])
    return a

def integral(a, round_constants):
    """
    Generate a list of inputs for the known key distinguisher.
    """
    inputs = []
    
    for x in range(p_mini):
        inputs.append(f_mini_inv(a*x, round_constants[:2]))
    return inputs

def test_sums(inputs, round_constants):
    """
    Compute the sums of inputs and outputs.
    """
    sum_in  = zero_vector(Fp_mini, 16)
    sum_out = zero_vector(Fp_mini, 16)
    
    for input_ in inputs:
        sum_out += f_mini(input_, round_constants)
        
        add_in = input_
        for i in range(4):
            add_in[i] = S_mini(add_in[i])
        for i in range(4, 16):
            add_in[i] = T_mini(add_in[i])
        sum_in += add_in
    return sum_in, sum_out


# Tests

## Step by step test

In [ ]:
## Layer by layer test

input_vector = vector(Fp, range(16))

# Input: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
print("Input vector : ")
print(input_vector)


S_box = zero_vector(Fp, 16)
for i in range(4):
    S_box[i] = S(input_vector[i])
for i in range(4,16):
    S_box[i] = T(input_vector[i])

# Sbox: [0, 1, 8, 27, 
#        16384, 78125, 279936, 823543, 
#        2097152, 4782969, 10000000, 19487171, 
#        35831808, 62748517, 105413504, 170859375]
print("S-box output : ")
print(S_box)

MDS = M*S_box

# MDS: [7205364737005, 12042395183376, 12272828223119, 11340812806600, 
#       17092227296585, 16460982307488, 12559497484731, 17742389185208, 
#       13715383884189, 15300900766312, 14022676374143, 12105605573328, 
#       16882252994553, 11429837802656, 11703409729659, 14542940950688]
print("MDS output : ")
print(MDS)


## Round by round test

In [ ]:
input_vector = vector(Fp, range(16))

# Input: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
print("Input vector : ")
print(input_vector)

round_vec = f_round(input_vector, round_constants[0])
print("Round 0:")
print(round_vec)

for i in range(1, 5):
    round_vec = f_round(round_vec, round_constants[i])
    print("Round {}:".format(i))
    print(round_vec)
# Round 0, [13630782508720194763, 16896939616488417250, 10379461926478353614, 1965419705225900095, 
#           15232556039317481696, 15892650859074054562, 3989146699522356499, 2851429654516916073, 
#           8709150154677642965, 3694873970563706046, 12692454266991701284, 10722328271963650077, 
#           12745446202694634001, 17932435653561793077, 7558114238277667122, 15551061978796482092]
# Round 1, [5525363112442782795, 3196671190848592148, 6751865969657517721, 6246363638275815365, 
#           17526105136308566870, 6414711837228726335, 17165555941248860828, 10245588381698521362, 
#           17368540314743694648, 14208333065255843385, 12223130565514930992, 6796342552837496564, 
#           3963709218487736004, 11869507492304374432, 7491219276692511038, 17108779980408342790]
# Round 2, [6286105364352512153, 17985860294796311537, 9345400983058519278, 4823584712699818929, 
#           7592681812112444861, 12285819859301081349, 8696939243258074783, 12059135831042518743, 
#           6827571267333842278, 5013808258607437272, 12075345740667337345, 11971779355482478795, 
#           10695815297893868736, 11982168343789428883, 5106944887466967556, 6814426455101604673]
# Round 3, [7125984701079541885, 13703424305218035968, 15382640950439390211, 13892416975492464765, 
#           10013294237979062072, 12926974615054989023, 8765320937915539373, 5249117573702297764, 
#           3064463953341252716, 1630229434993870734, 9905993257711309839, 15255305736124280041, 
#           12189914890971570438, 18428914739834411467, 15996699576317910304, 11053828540506304354]
# Round 4, [14273019456630489802, 12225354657803044645, 18223679466392555512, 4879234115918641111, 
#           198243361942729835, 6697571774370475124, 3935892719377798608, 2781322532457452310, 
#           7475933807446249354, 7334965145562953054, 1275437117587945070, 2445375571864276273, 
#           17005006372293520413, 9537835648539327419, 12703602725074524970, 5428520427373770602]


## Collision attack on R=4, c=5.

### Write the system for $\vec{a}$

In [ ]:
# We choose to fix the values of the wires 0,1,2,3 (before S-boxes) and 5,6 (before power maps).
indices_b = [0,1,2,3,5,6]
indices_a = complementary(indices_b, 16)

# a is the vector before the S-box layer of the third round.
a = orthogonal_vector(M, indices_a)

# generate the system of equation to solve in order to fix the wires of the S-boxes of the third round.
I = ideal(K, list(a[:4]))
print(I)

### Solution of the system.

In [ ]:
# using MAGMA, we find the vector:
a = a(2857675761863926834, 12096320840878172146, 16939308632471610364, 506223986432544699)
print("State of a before round 3:")
print(a)

vec = M.inverse()*a

for i in range(4):
    vec[i] = S_inv(vec[i])
for i in range(4,16):
    vec[i] = T_inv(vec[i])

print("State of a before round 2:")
print(vec)
print("State of a in the input:")
print(M.inverse()*vec)

In [ ]:
# We can control the value of 5 wires, and have one degree of freedom left.
b = fix_constants(M, round_constants[0], round_constants[1], indices_b, vector(Fp, [1,2,3,4,5,6]))

print("State of b before round 3:")
print(b)

vec = f_round_inv(b, round_constants[1])
print("State of b before round 2:")
print(vec)

print("State of b in the input:")
print(M.inverse()*(vec-round_constants[0]))

### Testing the attack.

In [ ]:
# For all value of the capacity, we can fill the rate in order to have the output in some hyperplane.
target = [42,69,36,15,666]

x = attack_4_rounds(M, round_constants, a, indices_b, target)

print("The input x is:")
print(x)

y = f(x, round_constants[:4])

print("The output y is:")
print(y)

orth_vec = M[:5, :4].left_kernel().basis()[0]
res = orth_vec.dot_product(y[:5])

print("Check that the output lies in the right hyperplane:")
print(res)


## Full round integral distinguisher (on Tip5 mini)

In [ ]:
# Compute the vector to avoid split-and-lookups.
a = get_a(M_mini)

# Compute the inputs and outputs
inputs = integral(a, round_constants_mini)
sum_in, sum_out = test_sums(inputs, round_constants_mini)

print("Inputs :")
print("Sum S(x) = {}".format(sum_in))

print("Outputs :")
print("M^-1 * Sum f(x) = {}".format(M_mini.inverse()*sum_out))